In [53]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [54]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-10-08 20:04:57--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.6’

book-crossings.zip. 100%[===================>]  24.88M   166MB/s    in 0.2s    

2025-10-08 20:04:57 (166 MB/s) - ‘book-crossings.zip.6’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: b
error:  invalid response [b]
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [55]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [56]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [57]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [58]:
book_counts = df_ratings['isbn'].value_counts()
valid_books = book_counts[book_counts >= 100].index

user_counts = df_ratings['user'].value_counts()
valid_users = user_counts[user_counts >= 200].index
df_ratings_filtered = df_ratings[
    df_ratings['isbn'].isin(valid_books) &
    df_ratings['user'].isin(valid_users)
].reset_index(drop=True)


In [59]:
merged_df = pd.merge(df_ratings_filtered, df_books, on='isbn')
merged_df = merged_df[['isbn', 'title', 'user', 'rating']]
# merged_df = merged_df.drop_duplicates(subset='title', keep='first').reset_index(drop=True)
display(merged_df.head())

,isbn,title,user,rating
0,002542730X,Politically Correct Bedtime Stories: Modern Ta...,277427,10.0
1,0060930535,The Poisonwood Bible: A Novel,277427,0.0
2,0060934417,Bel Canto: A Novel,277427,0.0
3,0061009059,One for the Money (Stephanie Plum Novels (Pape...,277427,9.0
4,0140067477,The Tao of Pooh,277427,0.0


In [44]:
book_user_matrix = merged_df.pivot_table(
    index='title',
    columns='user',
    values='rating',
    fill_value=0
)
model_knn = NearestNeighbors(
    n_neighbors=6,
    metric='cosine',
    algorithm='brute'
)
model_knn.fit(book_user_matrix.values)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [47]:
def get_recommends(book_title):
    if book_title not in book_user_matrix.index:
        return [book_title, []]

    book_idx = book_user_matrix.index.get_loc(book_title)
    book_vector = book_user_matrix.iloc[book_idx].values.reshape(1, -1)

    distances, indices = model_knn.kneighbors(book_vector)

    rec_books = []
    for dist, idx in zip(distances.flatten()[1:], indices.flatten()[1:]):
        rec_title = book_user_matrix.index[idx]
        rec_books.append([rec_title, float(dist)])
    rec_books = rec_books[::-1]
    return [book_title, rec_books]

In [60]:
print(get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))"))

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['Catch 22', 0.793983519077301], ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], ['Interview with the Vampire', 0.7345068454742432], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787], ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178411602973938]]]


In [61]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]
You passed the challenge! 🎉🎉🎉🎉🎉
